In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
sc = SparkContext.getOrCreate()

## BagofWords

In [11]:
def bagOfWords(texto):
    texto= re.sub(r'[^A-Za-z0-9 ]', '', texto).strip().lower()
    texto=texto.split(" ")

    auxRDD=sc.parallelize(texto)
    auxRDD= (auxRDD.
             map(lambda x:[x,1]).
             reduceByKey(lambda x,y:x+y).
             map(lambda x:(x[0],x[1]) )
            )
    return auxRDD.collect()
bagOfWords("me gusta el tangananika , tangananika nika nika nika")

[('el', 1),
 ('tangananika,', 1),
 ('me', 1),
 ('tangananika', 1),
 ('gusta', 1),
 ('nika', 3)]

## Ngrams

In [45]:
def ngramsTexto(n,texto):
    texto=texto.split(" ")
    tamanio=len(texto)
    arreglo=[]
    for i in range (tamanio-1):
        arreglo.append(str(texto[i]+" "+texto[i+1]))
    return arreglo

    
def ngramsDatos(n,Datos):
    auxRDD=sc.parallelize(Datos)
    auxRDD=auxRDD.map(lambda x:ngramsTexto(n,x))
    return auxRDD.collect()
ngrams(2,"el pato juan era bien pata de pato tato")

['el pato',
 'pato juan',
 'juan era',
 'era bien',
 'bien pata',
 'pata de',
 'de pato',
 'pato tato']

## nSkipGrams

In [47]:
def nskipGramsTexto(n,k,texto):
    texto=texto.split(" ")
    tamanio=len(texto)
    arreglo=[]
    for i in range (tamanio-k):
        arreglo.append(str(texto[i]+" "+texto[i+k]))
    return arreglo
def nskipGramsDatos(n,Datos):
    auxRDD=sc.parallelize(Datos)
    auxRDD=auxRDD.map(lambda x:nskipGramsTexto(n,k,x))
    return auxRDD.collect()
nskipGrams(2,3,"el pato juan era bien pata de pato tato")

['el era', 'pato bien', 'juan pata', 'era de', 'bien pato', 'pata tato']

## TF.IDF

In [51]:
import math
def TFPorDocumento(documento):
    documento=documento.split(" ")
    cantidadTotalDoc=len(documento)
    auxRDD=sc.parallelize(documento)
    auxRDD= (auxRDD.
             map(lambda x:[x,1]).
             reduceByKey(lambda x,y:x+y).
             map(lambda x:(x[0],x[1],1+math.log10(x[1]/cantidadTotalDoc)) )
            )
    return auxRDD.collect()
TFPorDocumento("juanito juanito paco paco pato el feo")

[('el', 1, 0.1549019599857432),
 ('feo', 1, 0.1549019599857432),
 ('paco', 2, 0.45593195564972433),
 ('juanito', 2, 0.45593195564972433),
 ('pato', 1, 0.1549019599857432)]

In [52]:
def IDFCorpus(corpus):
    nroTotalDoc=len(corpus)
    auxRDD=sc.parallelize(corpus)
    auxRDD=(auxRDD.
            flatMap(lambda x:list(set(x.split(" ")))).
             map(lambda x:[x,1]).
             reduceByKey(lambda x,y:x+y).
             map(lambda x:(x[0],x[1],math.log10(1+x[1]/nroTotalDoc)))

            )
    
    aux=auxRDD.collect()
    diccionario={}
    for i in aux: diccionario[i[0]]=i[2]
    return diccionario
    #return corpusRDD.collect()
IDFCorpus(["juanito juanito paco paco pato el feo","feo es ser llamado paco"])

{'el': 0.17609125905568124,
 'feo': 0.3010299956639812,
 'paco': 0.3010299956639812,
 'juanito': 0.17609125905568124,
 'llamado': 0.17609125905568124,
 'es': 0.17609125905568124,
 'pato': 0.17609125905568124,
 'ser': 0.17609125905568124}

In [53]:
def TFIDF(corpus):
    idf=IDFCorpus(corpus)
    for i in range (len(corpus)):
        print("===documento ",i,"====")
        TF=TFPorDocumento(corpus[i])
        for palabra in TF:
            print(palabra[0],palabra[2]*idf[palabra[0]])
TFIDF(["juanito juanito paco paco pato el feo","feo es ser llamado paco"])

===documento  0 ====
el 0.02727688116408227
feo 0.04663013634285046
paco 0.13724919463230698
juanito 0.08028563211407898
pato 0.02727688116408227
===documento  1 ====
feo 0.09061905828945656
paco 0.09061905828945656
llamado 0.05300875094999672
es 0.05300875094999672
ser 0.05300875094999672
